In [1]:
import os
os.chdir(os.path.expanduser("~/Projects/zimmer"))
import copy
import pickle

import numpy as np
np.random.seed(1234)

import matplotlib.pyplot as plt
plt.ion()

from tqdm import tqdm
from functools import partial

from zimmer.io import load_kato_data
import zimmer.plotting as zplt
from zimmer.observations import HierarchicalIndependentAutoRegressiveObservations, HierarchicalAutoRegressiveObservations 
from zimmer.transitions import HierarchicalRecurrentTransitions, HierarchicalRecurrentOnlyTransitions
from zimmer.util import cached

from ssm.models import HMM
from ssm.core import _HMM
from ssm.init_state_distns import InitialStateDistribution
from ssm.transitions import RecurrentTransitions, InputDrivenTransitions, StationaryTransitions, \
    NeuralNetworkRecurrentTransitions, RecurrentOnlyTransitions
from ssm.observations import IndependentAutoRegressiveObservations

from ssm.util import find_permutation, compute_state_overlap
from ssm.preprocessing import pca_with_imputation, trend_filter, standardize

In [2]:
# Specify outputs
results_dir = "results/kato/2018-08-29"

In [3]:
# Load the data
ys, ms, z_trues, z_true_key, neuron_names = load_kato_data(include_unnamed=False, signal="dff")
ys = [trend_filter(y) for y in ys]


K_true = len(z_true_key)
N = ys[0].shape[1]
W = len(ys)
Ts = [y.shape[0] for y in ys]

Only including named neurons.
59 neurons across all 5 worms


In [4]:
D = 10   # dimensionality of continuous latent states
M = 0   # dimensionality of input

# Dimensionality reduction on $\Delta$F/F 

In [5]:
# Run PCA to get a 3d projection of the data
from scipy.ndimage import gaussian_filter1d
from ssm.preprocessing import pca_with_imputation
pca, xs = pca_with_imputation(D, ys, ms)
# xs = [gaussian_filter1d(x, 1, axis=0) for x in xs]
# dxs = [gaussian_filter1d(np.gradient(x, axis=0), 1, axis=0) for x in xs]

lim = 1.1 * abs(np.vstack(xs)).max()

In [6]:
# plt.figure(figsize=(12, 16))
# for w, (x, z) in enumerate(zip(xs, z_trues)):
#     for d in range(1, D):
#         ax = plt.subplot(D, W, (d-1) * W + w+1)
#         zplt.plot_2d_continuous_states(x, z, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, d), ax=ax)
#         plt.ylabel("PC {}".format(d+1) if w == 0 else "")
#         plt.title("worm {}".format(w+1))

# plt.suptitle("Continuous Latent States (Zimmer Labels)")
# plt.tight_layout()

# Split into train/test/validation

In [7]:
chunk = 250
train_frac = 0.7
val_frac = 0.15
all_ys = []
all_xs = []
all_ms = []
all_tags = []
all_z_trues = []
all_choices = []
for tag, (y, x, m, ztr) in enumerate(zip(ys, xs, ms, z_trues)):
    T = y.shape[0]
    C = 0
    for start in range(0, T, chunk):
        stop = min(start+chunk, T)
        all_ys.append(y[start:stop])
        all_xs.append(x[start:stop])
        all_ms.append(m[start:stop])
        all_z_trues.append(ztr[start:stop])
        all_tags.append(tag)
        C += 1
        
    # assign some of the data to train, val, and test
    choices = -1 * np.ones(C)
    choices[:int(train_frac * C)] = 0
    choices[int(train_frac * C):int((train_frac + val_frac) * C)] = 1
    choices[int((train_frac + val_frac) * C):] = 2
    choices = choices[np.random.permutation(C)]
    all_choices.append(choices)

all_choices = np.concatenate(all_choices)
get = lambda arr, chc: [x for x, c in zip(arr, all_choices) if c == chc]

train_ys = get(all_ys, 0)
train_xs = get(all_xs, 0)
train_ms = get(all_ms, 0)
train_zs = get(all_z_trues, 0)
train_tags = get(all_tags, 0)

val_ys = get(all_ys, 1)
val_xs = get(all_xs, 1)
val_ms = get(all_ms, 1)
val_zs = get(all_z_trues, 1)
val_tags = get(all_tags, 1)

test_ys = get(all_ys, 2)
test_xs = get(all_xs, 2)
test_ms = get(all_ms, 2)
test_zs = get(all_z_trues, 2)
test_tags = get(all_tags, 2)

In [8]:
print(train_tags)
print(val_tags)
print(test_tags)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[0, 0, 1, 1, 2, 2, 3, 3, 4, 4]
[0, 0, 1, 1, 2, 2, 3, 3, 4, 4]


# Fit an ARHMM to the continuous latent states, sweeping over number of discrete latent states

In [9]:
Ks = np.arange(2, 21, step=2)
transitionss = ["recurrent"]
observationss = ["ar"]

In [ ]:
def _fit_hmm(K, transitions, observations):
    hmm = HMM(K, D, M, transitions=transitions, observations=observations)
    lps = hmm.fit(train_xs, method="em", num_em_iters=100)
    val_ll = hmm.log_likelihood(val_xs)
    return hmm, lps, val_ll

In [ ]:
hmm_results = {}
for K in Ks:
    for transitions in transitionss:
        for observations in observationss:
            hmm_results_name = "hmm_{}_{}_K{}".format(transitions, observations, K)
            fit = cached(results_dir, hmm_results_name)(_fit_hmm)
            
            print("Fitting ", hmm_results_name)
            hmm_results[hmm_results_name] = fit(K, transitions, observations)

Fitting  hmm_recurrent_ar_K2
Fitting  hmm_recurrent_ar_K4
Fitting  hmm_recurrent_ar_K6
Iteration 0.  LP: 181209.9
Iteration 1.  LP: 185224.4
Iteration 2.  LP: 185984.6
Iteration 3.  LP: 186076.7
Iteration 4.  LP: 186045.6
Iteration 5.  LP: 185958.0
Iteration 6.  LP: 185923.9
Iteration 7.  LP: 185945.2
Iteration 8.  LP: 185994.7
Iteration 9.  LP: 186058.3
Iteration 10.  LP: 186079.4
Iteration 11.  LP: 186088.5
Iteration 12.  LP: 186156.7
Iteration 13.  LP: 186186.9
Iteration 14.  LP: 186221.3
Iteration 15.  LP: 186279.3
Iteration 16.  LP: 186357.4
Iteration 17.  LP: 186420.9
Iteration 18.  LP: 186473.5
Iteration 19.  LP: 186527.6
Iteration 20.  LP: 186558.1
Iteration 21.  LP: 186587.5
Iteration 22.  LP: 186620.1
Iteration 23.  LP: 186651.6
Iteration 24.  LP: 186676.5
Iteration 25.  LP: 186676.8
Iteration 26.  LP: 186653.8
Iteration 27.  LP: 186643.0
Iteration 28.  LP: 186596.1
Iteration 29.  LP: 186518.8
Iteration 30.  LP: 186497.9
Iteration 31.  LP: 186474.2
Iteration 32.  LP: 186461.3

# Fit hierarchical HMMs

In [ ]:
def _fit_hierarchical_hmm(K, transitions, observations, hmm):
    # Construct the HMM components
    init_state_distn = InitialStateDistribution(K, D, M)

    transition_classes = dict(recurrent=HierarchicalRecurrentTransitions)
    transition_distn = transition_classes[transitions](K, D, W, M, eta=1e-4)
    
    observation_classes = dict(ar=HierarchicalAutoRegressiveObservations, 
                               independent_ar=HierarchicalIndependentAutoRegressiveObservations)
    observation_distn = observation_classes[observations](K, D, W, M, eta=1e-4)
    
    # Construct the HMM
    hhmm = _HMM(K, D, M, init_state_distn, transition_distn, observation_distn)

    # Initialize with the standard HMM
    hhmm.init_state_distn.params = copy.deepcopy(hmm.init_state_distn.params)
    hhmm.transitions.initialize_from_standard(hmm.transitions)
    hhmm.observations.initialize_from_standard(hmm.observations)

    # Fit
    lps = hhmm.fit(train_xs, tags=train_tags, method="em", num_em_iters=100, initialize=False)
    
    # Validate
    val_ll = hhmm.log_likelihood(val_xs, tags=val_tags)
    return hhmm, lps, val_ll

In [ ]:
hhmm_results = {}
for K in Ks:
    for transitions in transitionss:
        for observations in observationss:
            # Get the HMM results
            hmm_results_name = "hmm_{}_{}_K{}".format(transitions, observations, K)
            hmm, _, _ = hmm_results[hmm_results_name]
            
            # Fit the Hierarchical HMM
            hhmm_results_name = "hhmm_{}_{}_K{}".format(transitions, observations, K)
            fit = cached(results_dir, hhmm_results_name)(_fit_hierarchical_hmm)
            print("Fitting ", hhmm_results_name)
            hhmm_results[hhmm_results_name] = fit(K, transitions, observations, hmm)

In [ ]:
# Plot train and validation likelihoods
plt.figure(figsize=(6, 3))
hmm_train_lls = [hmm_results["hmm_recurrent_ar_K{}".format(K)][1][-1] for K in Ks]
hhmm_train_lls = [hhmm_results["hhmm_recurrent_ar_K{}".format(K)][1][-1] for K in Ks]
plt.plot(Ks, hmm_train_lls, '-o', label="HMM")
plt.plot(Ks, hhmm_train_lls, '-o', label="HHMM")
plt.xlabel("K")
plt.ylabel("Validation LL")

plt.figure(figsize=(6, 3))
hmm_val_lls = [hmm_results["hmm_recurrent_ar_K{}".format(K)][2] for K in Ks]
hhmm_val_lls = [hhmm_results["hhmm_recurrent_ar_K{}".format(K)][2] for K in Ks]
plt.plot(Ks, hmm_val_lls, '-o', label="HMM")
plt.plot(Ks, hhmm_val_lls, '-o', label="HHMM")
plt.xlabel("K")
plt.ylabel("Validation LL")
plt.legend(loc="lower right")

In [ ]:
assert False

# Look at inferred state segmentations

In [ ]:
z_infs = [hhmm.most_likely_states(x, tag=tag) for tag, x in enumerate(xs)]
hhmm.permute(find_permutation(np.concatenate(z_trues), np.concatenate(z_infs)))
z_infs = [hhmm.most_likely_states(x, tag=tag) for tag, x in enumerate(xs)]

In [ ]:
np.bincount(np.concatenate(z_infs), minlength=8)

In [ ]:
zplt.plot_latent_trajectories_vs_time(xs, z_infs, plot_slice=(0, 1000))

In [ ]:
plt.figure(figsize=(12, 3))
for w, (x, z) in enumerate(zip(xs, z_infs)):
    ax = plt.subplot(1, W, w+1)
    zplt.plot_2d_continuous_states(x, z, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, 1), ax=ax)
    plt.ylabel("PC 2" if w == 0 else "")
    plt.xlabel("PC 1")
    plt.title("worm {}".format(w+1))
    

In [ ]:
plt.figure(figsize=(12, 3))
for w, (x, z) in enumerate(zip(xs, z_infs)):
    ax = plt.subplot(1, W, w+1)
    zplt.plot_2d_continuous_states(x, z, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, 2), ax=ax)
    plt.ylabel("PC 3" if w == 0 else "")
    plt.xlabel("PC 1")

In [ ]:
if D > 3:
    plt.figure(figsize=(12, 3))
    for w, (x, z) in enumerate(zip(xs, z_infs)):    
        ax = plt.subplot(1, W, w+1)
        zplt.plot_2d_continuous_states(x, z, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, 3), ax=ax)
        plt.ylabel("PC 4" if w == 0 else "")
        plt.xlabel("PC 1")    
        plt.title("worm {}".format(w+1))

In [ ]:
if D > 4:
    plt.figure(figsize=(12, 3))
    for w, (x, z) in enumerate(zip(xs, z_infs)):
        ax = plt.subplot(1, W, w+1)
        zplt.plot_2d_continuous_states(x, z, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, 4), ax=ax)
        plt.ylabel("PC 5" if w == 0 else "")
        plt.xlabel("PC 1")    
        plt.title("worm {}".format(w+1))  

In [ ]:
if D > 5:
    plt.figure(figsize=(12, 3))
    for w, (x, z) in enumerate(zip(xs, z_infs)):  
        ax = plt.subplot(1, W, w+1)
        zplt.plot_2d_continuous_states(x, z, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, 5), ax=ax)
        plt.ylabel("PC 6" if w == 0 else "")
        plt.xlabel("PC 1")    
        plt.title("worm {}".format(w+1))

    plt.suptitle("Continuous Latent States (Zimmer Labels)")

# Compare inferred and manually labeled states

In [ ]:
zplt.plot_state_overlap(z_infs, z_trues)

In [ ]:
# Helper function to find overlap percentages
def compute_pct_overlap(zi, ztr):
    overlap = np.zeros((K, K_true))
    for k in range(K):
        overlap[k] = np.bincount(ztr[zi == k], minlength=K_true).astype(float)
        overlap[k] /= (overlap[k].sum() + 1e-3)
    return overlap

# Find a permutation so that the bar codes look progressive
total_overlap = compute_pct_overlap(np.concatenate(z_infs), np.concatenate(z_trues))
overlap_perm = np.argsort(np.argmax(total_overlap, axis=1))

# Helper function to plot "barcodes"
def plot_overlap_barcode(ax, overlap):
    for i,k in enumerate(overlap_perm):        
        for ktr in range(K_true):
            plt.bar(i, overlap[k, ktr], bottom=np.sum(overlap[k, :ktr]), color=zplt.default_colors[ktr], width=0.8)
    ax.set_xlim(-.5, K-.5)
    
# Plot all overlaps as bar codes
plt.figure(figsize=(12, 4))

# Plot the total overlap first
ax = plt.subplot(1, W+1, 1)
plot_overlap_barcode(ax, total_overlap)
plt.ylabel("Pct of manual state")
plt.yticks([0, .25, .5, .75, 1], [0, 25, 50, 75, 100])
plt.ylim(0, 1)
plt.xlabel("Inferred state")
plt.xticks(np.arange(K), np.arange(K)+1)
plt.title("All worms")

for w in range(W):
    ax = plt.subplot(1, W+1, w+2)
    overlap_w = compute_pct_overlap(z_infs[w], z_trues[w])
    plot_overlap_barcode(ax, overlap_w)
    plt.yticks([])        
    plt.ylim(0, 1)
    plt.xlabel("Inferred state")
    plt.xticks(np.arange(K), np.arange(K)+1)
    plt.title("Worm {}".format(w+1))
plt.tight_layout()

# Print key
for color_name, state_name in zip(zplt.color_names, z_true_key):
    print("{} : {}".format(color_name, state_name))

# Simulate from an HMM, reducing the dynamics noise

In [ ]:
hhmm_low_noise = copy.deepcopy(hhmm)
hhmm_low_noise.observations.inv_sigmas -= 2

In [ ]:
pad = 3
Tsmpl = Ts[0]
i = 1
zpre, xpre = z_infs[i][-pad:], xs[i][-pad:]
zsmpl, xsmpl = hhmm_low_noise.sample(Tsmpl, prefix=(zpre, xpre), tag=i, with_noise=True)

zfull = np.concatenate((zpre, zsmpl))
xfull = np.concatenate((xpre, xsmpl))

In [ ]:
plt.figure(figsize=(12, 6))
ax = plt.subplot(1, 2, 1, aspect="equal")
zplt.plot_2d_continuous_states(xfull, zfull, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, 1), ax=ax)
plt.plot(xfull[pad-1,0], xfull[pad-1,1], 'k*')
plt.xlabel("PC 1")
plt.ylabel("PC 2")

ax = plt.subplot(1, 2, 2, aspect="equal")
zplt.plot_2d_continuous_states(xfull, zfull, xlims=(-lim, lim), ylims=(-lim, lim), inds=(0, 2), ax=ax)
plt.plot(xfull[pad-1,0], xfull[pad-1,2], 'k*')
plt.xlabel("PC 1")
plt.ylabel("PC 3")

In [ ]:
plt.figure(figsize=(12, 8))
ysim = xfull.dot(pca.components_) + pca.mean_
plt.plot(np.arange(ysim.shape[0]) / 3.0, ysim - np.arange(N), '-k')
plt.yticks(-np.arange(N), neuron_names)
plt.ylim(-N,1)
plt.xlim(0, ysim.shape[0] / 3.0)
plt.xlabel("time (s)")

In [ ]:
# Plot real data for comparison
plt.figure(figsize=(12, 8))
plt.plot(np.arange(Ts[0]) / 3.0, ys[0] - np.arange(N), '-k')
plt.yticks(-np.arange(N), neuron_names)
plt.ylim(-N,1)
plt.xlim(0, Ts[0] / 3.0)
plt.xlabel("time (s)")